<a href="https://colab.research.google.com/github/otavioqs16/breast_cancer-ia/blob/main/breast_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import torch
import numpy as np

In [41]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid() 
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        output = self.sigmoid(output)
        return output

In [42]:
from sklearn import datasets

In [ ]:
breast_cancer = datasets.load_breast_cancer()
print(breast_cancer)

In [ ]:
dados = breast_cancer.data
classes = breast_cancer.target
nome_classes = breast_cancer.target_names
print(dados)
print(classes)
print(nome_classes)

In [ ]:
print(dados.shape)
for i in range(len(dados[1])):
  print(np.max(dados[i]))
  max = np.max(dados[i])
  if max < 10:
    dados[i] /= 10
  elif max < 100:
    dados[i] /= 100
  elif max < 1000:
    dados[i] /= 1000
  else:
    dados[i] /= 10000
for i in range(len(dados[1])):
  print(np.max(dados[i]))


In [ ]:
from sklearn.utils import shuffle
x = dados
y = classes
x, y = shuffle(x, y)
print(x, y)

In [ ]:
training_input = torch.FloatTensor(x[0:500, :])
training_output = torch.FloatTensor(y[0:500])
test_input = torch.FloatTensor(x[500:, :])
test_output = torch.FloatTensor(y[500:])
print(test_output.size())
print(test_input.size())
print(training_input)
print(training_output)
print(test_input)
print(test_output)

In [48]:
input_size = training_input.size()[1]
hidden_size = 2
model = Net(input_size, hidden_size)
#print(model)
for param in model.parameters():
  #print(param)
  pass

In [ ]:
#Preparando o treinamento
criterion = torch.nn.MSELoss() #Mean Square Error Loss (saida_real - saida_prevista)^2
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

#Avaliar o erro inicial da rede
model.eval() # roda a rede mas sem atualizar os pesos
y_pred = model(test_input)
before_train = criterion(y_pred.squeeze(), test_output)
#[100,] --> [100]
print(f'Erro antes do treinamento: {before_train.item()}')

In [ ]:
#Treinamento da rede
model.train()
epochs = 400001
errors = []
for epoch in range(epochs):
  optimizer.zero_grad()
  #fazer o forward da minha rede
  y_pred = model(training_input)
  #calcula o erro desse treinamento
  loss = criterion(y_pred.squeeze(), training_output)
  errors.append(loss.item())
  if epoch % 1000 == 0:
    print(f'Epoch: {epoch}. Train loss: {loss.item()}')
  #backpropagation
  loss.backward()
  optimizer.step()

In [ ]:
#Avaliar o erro pós treinamento da rede
model.eval() # roda a rede mas sem atualizar os pesos
y_pred = model(test_input)
after_train = criterion(y_pred.squeeze(), test_output)
print(f'Erro pós-treinamento: {after_train.item()}')

In [52]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def plotcharts(errors):
    errors = np.array(errors)
    plt.figure(figsize=(12, 5))
    graf02 = plt.subplot(1, 2, 1) # nrows, ncols, index
    graf02.set_title('Errors')
    plt.plot(errors, '-')
    plt.xlabel('Epochs')
    graf03 = plt.subplot(1, 2, 2)
    graf03.set_title('Tests')
    a = plt.plot(test_output.numpy(), 'yo', label='Real')
    plt.setp(a, markersize=10)
    a = plt.plot(y_pred.detach().numpy(), 'b+', label='Predicted')
    plt.setp(a, markersize=10)
    plt.legend(loc=7)
    plt.show()
plotcharts(errors)